In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
# %qtconsole

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

from dask.distributed import Client, LocalCluster
import dask
import logging

logging.basicConfig(level=logging.INFO)

client = Client(processes=False)

client

/Users/edeno/miniconda3/envs/replay_trajectory_paper/lib/python3.6/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://168.122.203.7/58346/1 Dashboard: http://localhost:63389/status,Cluster Workers: 1 Cores: 4 Memory: 17.18 GB


In [3]:
from src.load_data import load_data

epoch_key = ('bon', 3, 2)

data = load_data(epoch_key)

INFO:src.load_data:Finding ripple times...


In [5]:
from src.parameters import replay_speed, place_bin_size, movement_var, discrete_diag
from replay_trajectory_classification import SortedSpikesClassifier

position = data['position_info'].linear_position2
track_labels = data['position_info'].arm_name
ripple_times = data['ripple_times'].loc[:, ['start_time', 'end_time']]

continuous_transition_types = (
    [['w_track_1D_random_walk', 'uniform', 'identity'],
     ['uniform',   'uniform', 'uniform'],
     ['w_track_1D_random_walk', 'uniform', 'identity']])

classifier = SortedSpikesClassifier(
    place_bin_size=0.5, movement_var=movement_var,
    replay_speed=1,
    discrete_transition_diag=discrete_diag,
    spike_model_penalty=0.5, knot_spacing=10,
    continuous_transition_types=continuous_transition_types).fit(
    position, data['spikes'], track_labels=track_labels)

INFO:replay_trajectory_classification.classifier:Fitting initial conditions...
INFO:replay_trajectory_classification.classifier:Fitting state transition...
INFO:replay_trajectory_classification.classifier:Fitting place fields...


In [ ]:
results = classifier.predict(data['spikes'], time=data['position_info'].index)